# Blcok Recognition

## Import Libraries

In [1]:
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import open3d as o3d
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Read Test Images

In [2]:
img_color = cv2.imread('test_imgs/jenga_tower_color_out1.png')
img_depth = cv2.imread('test_imgs/jenga_tower_depth_out1.png', 0)
print('color shape: ', img_color.shape)
print('depth shape: ', img_depth.shape)

color shape:  (1536, 2048, 3)
depth shape:  (1536, 2048)


## Extract Blocks' Masks By Colors

In [3]:
height, width = img_color.shape[:2] # 이미지의 높이와 너비 불러옴, 가로 [0], 세로[1]

img_hsv = cv2.cvtColor(img_color, cv2.COLOR_BGR2HSV) # cvtColor 함수를 이용하여 hsv 색공간으로 변환

In [4]:
colors = ['green', 'pink', 'yellow', 'blue', 'violet', 'red']

In [5]:
# RED
lower_red1 = np.array([0, 130, 50])
upper_red1 = np.array([15, 255, 255])
lower_red2 = np.array([160,130,50])
upper_red2 = np.array([179,255,255])

# PINK
lower_pink1 = np.array([0, 70, 80])
upper_pink1 = np.array([10, 130, 255])
lower_pink2 = np.array([150,70,80])
upper_pink2 = np.array([179,130,255])

# GREEN
lower_green = (70-20, 50, 50)
upper_green = (70+15, 255, 255)

# YELLOW
lower_yellow = (30-10, 80, 80)
upper_yellow = (30+10, 255, 255)

# BLUE
lower_blue = (100-10, 100, 100)
upper_blue = (100+9, 255, 255)

# VIOLET
lower_violet = (130-20, 50, 30)
upper_violet = (130+20, 255, 255)

In [6]:
blocks_rgb_by_color = []
blocks_mask_by_color = []
for color in colors:
    if color == 'pink' or color =='red':
        for i in (1,2):
            exec(f"lower_color{i} = lower_{color}{i}")
            exec(f"upper_color{i} = upper_{color}{i}")

        mask_color1 = cv2.inRange(img_hsv, lower_color1, upper_color1)
        mask_color2 = cv2.inRange(img_hsv, lower_color2, upper_color2)
        img_mask_color = mask_color1 + mask_color2

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        erosion_image_color = cv2.erode(img_mask_color, kernel, iterations=2)  #// make erosion image
        img_mask_color = cv2.dilate(erosion_image_color, kernel, iterations=2)  #// make dilation image

        # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
        img_result_color = cv2.bitwise_and(img_color, img_color, mask = img_mask_color) 
        
        exec(f"img_result_{color} = img_result_color")
    
    else:
        exec(f"lower_color = lower_{color}")
        exec(f"upper_color = upper_{color}")

        img_mask_color = cv2.inRange(img_hsv, lower_color, upper_color) # 범위내의 픽셀들은 흰색, 나머지 검은색

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        erosion_image_color = cv2.erode(img_mask_color, kernel, iterations=2)  #// make erosion image
        img_mask_color = cv2.dilate(erosion_image_color, kernel, iterations=2)  #// make dilation image

        # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
        img_result_color = cv2.bitwise_and(img_color, img_color, mask = img_mask_color) 

        exec(f"img_result_{color} = img_result_color")
        
    _, src_bin = cv2.threshold(img_mask_color, 0, 255, cv2.THRESH_OTSU)
    each_color_filtered = cv2.bitwise_and(img_color, img_color, mask = src_bin)
    
    # if color == 'blue':
    #     cv2.imshow('src_bin', src_bin)
    #     cv2.imshow(f'{color}_filtered', each_color_filtered)
    #     cv2.waitKey(0)
    #     cv2.destroyAllWindows()

    cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)

    blocks_color = []
    blocks_mask = []

    for i in range(1, cnt): # 각각의 객체 정보에 들어가기 위해 반복문. 범위를 1부터 시작한 이유는 배경을 제외
        (x, y, w, h, area) = stats[i]
        cen_x, cen_y = map(int, centroids[i])
        block_mask = (labels==i)*img_mask_color
        block_color = cv2.bitwise_and(img_color, img_color, mask = block_mask)
        
        # 노이즈 제거
        if area < 600:
            continue
        
        
        blocks_color.append(block_color)
        blocks_mask.append(block_mask)
        
        
    exec(f"blocks_rgb_{color} = blocks_color")
    exec(f"blocks_mask_{color} = blocks_mask")
    exec(f"blocks_rgb_by_color.append(blocks_rgb_{color})")
    exec(f"blocks_mask_by_color.append(blocks_mask_{color})")
    

In [7]:
tower_mask = 0
tower_color = 0
for mask, color in zip(blocks_mask_by_color, blocks_rgb_by_color):
    for block_m in mask:
        tower_mask += block_m
    
    for block_c in color:
        tower_color += block_c

In [8]:
cv2.imshow('tower mask', tower_mask)
# cv2.imwrite('./test_imgs/tower_mask.png', tower_mask)
cv2.imshow('tower color', tower_color)
# cv2.imwrite('./test_imgs/tower_color.png', tower_color)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
for c, b, rgb_m in zip(colors, blocks_mask_by_color, blocks_rgb_by_color):
    print(c)
    print(len(b), "Blocks")

green
6 Blocks
pink
6 Blocks
yellow
6 Blocks
blue
5 Blocks
violet
6 Blocks
red
6 Blocks


## Get PointCloud from RGB Image + Depth Image

In [10]:
# temp intrinsic matrix
intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.intrinsic_matrix = [[971.179, 0, 1025.07],[0, 970.984, 778.291],[0, 0, 1]]
intrinsic.intrinsic_matrix = [[971.179, 0, 1025.07],[0, 970.984, 778.291],[0, 0, 1]]

In [11]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=50, origin=[0, 0, 0])

In [12]:
def get_pointcloud_from_color_depth(color_image, depth_image, intrinsic):
    o3d_img = o3d.geometry.Image()
    
    if isinstance(color_image, type(o3d_img)):
        pass
    elif isinstance(color_image, np.ndarray):
        color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        color_image = o3d.geometry.Image(color_image)
        
    if isinstance(depth_image, type(o3d_img)):
        pass
    elif isinstance(depth_image, np.ndarray):
        depth_image = o3d.geometry.Image(depth_image)
        
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_image, depth_image)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsic)
    
    return pcd

In [13]:
masked_depth = cv2.bitwise_and(img_depth, img_depth, mask = tower_mask)

In [14]:
tower_pcd = get_pointcloud_from_color_depth(color_image=tower_color, depth_image=masked_depth, intrinsic=intrinsic)

In [15]:
print(tower_pcd)
print(np.asarray(tower_pcd.points))
o3d.visualization.draw_geometries([tower_pcd])

PointCloud with 99053 points.
[[ 3.60051560e-05 -1.69810720e-04  7.45098048e-04]
 [ 3.67723658e-05 -1.69810720e-04  7.45098048e-04]
 [ 3.75395756e-05 -1.69810720e-04  7.45098048e-04]
 ...
 [ 3.50973437e-05  1.22609574e-04  4.54901950e-04]
 [ 3.55657454e-05  1.22609574e-04  4.54901950e-04]
 [ 3.63447885e-05  1.23666560e-04  4.58823546e-04]]


In [16]:
blocks_pcd_by_color = []
all_pcd = []
for color, block_mask in zip(colors, blocks_mask_by_color):
    # print(color)
    blocks_pcd = []
    for msk in block_mask:
        masked_block_rgb = cv2.bitwise_and(tower_color, tower_color, mask = msk)
        masked_block_depth = cv2.bitwise_and(img_depth, img_depth, mask = msk)
        
        # Get Each Block's PointCloud
        pcd = get_pointcloud_from_color_depth(color_image=masked_block_rgb, depth_image=masked_block_depth, intrinsic=intrinsic)
        
        # Remove Outlier Points
        pcd, _ = pcd.remove_radius_outlier(512, 0.0001)
        blocks_pcd.append(pcd)
        all_pcd.append(pcd)
        
        # if color=='green':
        #     o3d.visualization.draw_geometries([pcd])
    
    exec(f"blocks_pcd_{color} = blocks_pcd")
    exec(f"blocks_pcd_by_color.append(blocks_pcd_{color})")

In [17]:
len(blocks_pcd_by_color)

6

## Tower ICP

In [26]:
initial_transform = np.asarray([[0, 0, -1, 0],
                        [1, 0, 0, 0],
                        [0, -1, 0, 0],
                        [0, 0, 0, 1]])

In [27]:
pcd_combined = o3d.geometry.PointCloud()
for point_id in range(len(all_pcd)):
    pcd_combined += all_pcd[point_id]

print(pcd_combined)
# print(len(pcd_combined.points))

o3d.visualization.draw_geometries([pcd_combined])

PointCloud with 97210 points.


## Blocks Transformation

In [28]:
trans = np.array([[ 5.14626774e-01, -8.57109340e-01, -2.28661932e-02, 2.42854632e+01],
                  [ 8.57406220e-01,  5.14323101e-01,  1.80643793e-02, -1.96452668e+00],
                  [-3.72253686e-03, -2.89020295e-02,  9.99575318e-01, 7.46547186e-01],
                  [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, 1.00000000e+00]])

In [29]:
resize = 763628.6545834081
move = np.array([388.72575451, -14.51926349,  96.9976991 ])

In [30]:
trans_1 = np.asarray([[0, 0, -1, 0],
                      [1, 0, 0, 0],
                      [0, -1, 0, 0],
                      [0, 0, 0, 1]])

In [31]:
def transform_blocks(pcd, icp_transform):
    pcd_temp = copy.deepcopy(pcd)
    pcd_temp.transform(initial_transform)
    aa = o3d.cpu.pybind.utility.Vector3dVector(np.array(pcd_temp.points)*resize + move)
    pcd_temp.points = aa
    pcd_temp.transform(icp_transform)
    
    return pcd_temp

In [32]:
# WHAT IS THE TARGET BLOCK?
target_block_color = 'violet'
target_block_label = 2

In [33]:
for col, pcds in zip(colors, blocks_pcd_by_color):
    if col != target_block_color:
        continue
    print(col)
    
    for idx, pcd in enumerate(pcds):
        # if idx != target_block_label:
        #     continue
        
        # print("--------------------------------")
        print(idx)
        pcd_new = transform_blocks(pcd, trans)
        
        # o3d.visualization.draw_geometries([pcd_new, mesh_frame, pcd_new.get_axis_aligned_bounding_box()])
        
        box_extent = pcd_new.get_axis_aligned_bounding_box().get_extent()
        print("BOX EXTENT : ", box_extent)
        
        center_coordinate = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points()).mean(axis=0)
        
        # print(np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points()))
        print("BOX CENTER COORDINATE : ", center_coordinate)
        
        # print("BOX MAX X,Y and MEAN Z Coordinate")
        x_max = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,0].max()
        y_max = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,1].max()
        z_mean = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,2].mean()
        
        if box_extent[1] > 70:
            print("PULL DIRECTION : X")
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_max - 25/2
            cen_y = y_max - 75/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x + 50
            target_y = cen_y
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        elif box_extent[0] > 70:
            print("PULL DIRECTION : Y")
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_max - 75/2
            cen_y = y_max - 25/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x
            target_y = cen_y + 50
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        elif abs(center_coordinate[0]) < 10 and box_extent [1] < 20:
            print("PUSH DIRECTION : Y or -Y")
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_max - 25/2
            cen_y = y_max - 75/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x
            target_y = cen_y + 150
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        elif abs(center_coordinate[1]) < 10 and box_extent [0] < 20:
            print("PUSH DIRECTION : X or -X")
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_max - 75/2
            cen_y = y_max - 25/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x + 150
            target_y = cen_y
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        elif box_extent[1] < 20:
            print("PULL DIRECTION : -X")
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_max - 25/2
            cen_y = y_max - 75/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x - 50
            target_y = cen_y
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        elif box_extent[0] < 20:
            print("PULL DIRECTION : -Y")
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_max - 75/2
            cen_y = y_max - 25/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x
            target_y = cen_y -50
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        mesh_COORD = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=center_coordinate)
        ptc1 = o3d.geometry.PointCloud()
        ptc1.points = pt1
        ptc1.paint_uniform_color([1, 0, 0])
        ptc2 = o3d.geometry.PointCloud()
        ptc2.points = pt2
        ptc2.paint_uniform_color([0, 0, 1])
        line = o3d.geometry.LineSet.create_from_point_cloud_correspondences(ptc1, ptc2, [(0,0)])
        o3d.visualization.draw_geometries([pcd_new, mesh_frame, pcd_new.get_axis_aligned_bounding_box(), ptc1, ptc2, line])

violet
0
BOX EXTENT :  [18.01380469 45.86354353 19.27738216]
BOX CENTER COORDINATE :  [ 31.11838134  -9.0417486  173.25224523]
PUSH DIRECTION : X or -X
-----BLOCK CENTER COORDINATE-----
X : 2.6252836826718706
Y : 1.3900231637957
Z : 173.2522452344987
-----TARGET COORDINATE-----
X : 152.62528368267186
Y : 1.3900231637957
Z : 173.2522452344987
----------------------------------
1
BOX EXTENT :  [ 3.54125868 24.29616531 14.67299635]
BOX CENTER COORDINATE :  [ 37.99772172   1.65238957 141.95342263]
PUSH DIRECTION : X or -X
-----BLOCK CENTER COORDINATE-----
X : 2.26835106057586
Y : 1.3004722231739745
Z : 141.95342263192975
-----TARGET COORDINATE-----
X : 152.26835106057587
Y : 1.3004722231739745
Z : 141.95342263192975
----------------------------------
2
BOX EXTENT :  [ 8.34718205 24.84551411 14.59065451]
BOX CENTER COORDINATE :  [ 35.86118868   1.70204927 112.93381929]
PUSH DIRECTION : X or -X
-----BLOCK CENTER COORDINATE-----
X : 2.5347797012837177
Y : 1.6248063295151898
Z : 112.9338192919